<a href="https://colab.research.google.com/github/vrhughes/DS4002-Project2/blob/main/SCRIPTS/TWFE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing
import pandas as pd
import numpy as np
import re

In [2]:
# Cloning the repo
! git clone https://github.com/vrhughes/DS4002-Project2

Cloning into 'DS4002-Project2'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 108 (delta 38), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (108/108), 635.12 KiB | 3.40 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [4]:
# Reading in data
#fr_analysis = pd.read_csv('/content/DS4002-Project2/OUTPUTS/fr_analysis_df.csv')
interp = pd.read_csv('/content/DS4002-Project2/OUTPUTS/interpolated_education_dataset.csv')
UN = pd.read_csv('/content/DS4002-Project2/DATA/fertility-rate-with-projections.csv')

In [5]:
# Getting dummies
df_encoded = pd.get_dummies(interp, columns=['Country', 'Year'])

In [9]:
df_encoded.head()
print(df_encoded.columns)

Index(['Fertility_Rate', 'GDP_Per_Capita', 'Homicide_Rate',
       'Avg_Yrs_Education', 'Infant_Mortality', 'Life_Expectancy',
       'Country_Bangladesh', 'Country_Chile', 'Country_Ethiopia',
       'Country_Iran', 'Country_Latvia', 'Country_Namibia', 'Country_Nauru',
       'Country_New Zealand', 'Country_Nicaragua', 'Country_Niger',
       'Country_Norway', 'Country_Panama', 'Country_South Korea',
       'Country_United States', 'Country_Venezuela', 'Country_World',
       'Year_1980', 'Year_1981', 'Year_1982', 'Year_1983', 'Year_1984',
       'Year_1985', 'Year_1986', 'Year_1987', 'Year_1988', 'Year_1989',
       'Year_1990', 'Year_1991', 'Year_1992', 'Year_1993', 'Year_1994',
       'Year_1995', 'Year_1996', 'Year_1997', 'Year_1998', 'Year_1999',
       'Year_2000', 'Year_2001', 'Year_2002', 'Year_2003', 'Year_2004',
       'Year_2005', 'Year_2006', 'Year_2007', 'Year_2008', 'Year_2009',
       'Year_2010', 'Year_2011', 'Year_2012', 'Year_2013', 'Year_2014',
       'Year_2015', 'Y

wow that's a lot of columns!

In [15]:
pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.7 MB/s eta 0:00:00


In [16]:
# Importing for TWFE
from matplotlib import style
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import graphviz as gr
from linearmodels.panel import PanelOLS

In [23]:
# Running linear regression on the covariates and the dummies
twfe = PanelOLS.from_formula("Fertility_Rate ~ GDP_Per_Capita+Homicide_Rate+Avg_Yrs_Education+Infant_Mortality+Life_Expectancy+EntityEffects+TimeEffects", data=interp.set_index(["Country", "Year"]))

result = twfe.fit(cov_type = 'clustered', cluster_entity = True, cluster_time = True)
result.summary.tables[1]

/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


,Parameter,Std. Err.,T-stat,P-value,Lower CI,Upper CI
GDP_Per_Capita,1.011e-05,3.136e-06,3.2240,0.0014,3.945e-06,1.627e-05
Homicide_Rate,0.0122,0.0079,1.5409,0.1241,-0.0034,0.0278
Avg_Yrs_Education,-0.4771,0.1624,-2.9385,0.0035,-0.7962,-0.1579
Infant_Mortality,0.1840,0.0920,2.0006,0.0461,0.0032,0.3648
Life_Expectancy,0.0411,0.0616,0.6675,0.5048,-0.0800,0.1622


In [25]:
# Getting coefficients
mod = smf.ols(formula='Fertility_Rate ~ GDP_Per_Capita+Homicide_Rate+Avg_Yrs_Education+Infant_Mortality+Life_Expectancy', data=interp).fit()
mod.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,11.7745,0.956,12.317,0.000,9.896,13.653
GDP_Per_Capita,1.966e-05,2.1e-06,9.352,0.000,1.55e-05,2.38e-05
Homicide_Rate,0.0075,0.004,2.019,0.044,0.000,0.015
Avg_Yrs_Education,-0.2365,0.023,-10.307,0.000,-0.282,-0.191
Infant_Mortality,0.0248,0.035,0.711,0.477,-0.044,0.094
Life_Expectancy,-0.1022,0.013,-7.929,0.000,-0.127,-0.077


In [ ]:
twfe_toy = df_encoded.assign(y_hat = twfe.fittedvalues)

plt.scatter(df_encoded.Year)

Try second